In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName('myApp').setMaster('local[*]'))

In [2]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

In [3]:
def step(item):
    cur_v, path, next_v = item[0], item[1][0], item[1][1]
    return (next_v, path+[next_v])

In [4]:
def complete(item):
    v, old_p, new_p = item[0], item[1][0], item[1][1]
    return (v, old_p if old_p is not None else new_p)

In [5]:
n = 400  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

In [6]:
start, end = 12, 34
paths = sc.parallelize([(start, [start])]).partitionBy(n)

while not paths.filter(lambda x: x[0] == end).count():
    paths = paths.join(forward_edges, n).map(step)

In [7]:
path = paths.filter(lambda x: x[0] == end).collect()[0][1]
print(','.join(map(str, path)))

12,422,53,52,107,20,23,274,34
